<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fecha de entrega: Mayo 27, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [1]:
import pandas as pd
import tweepy

# Leer las llaves
keys = pd.read_csv("mis_llaves.csv", sep=';',header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [2]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'],
                       lang='es',
                       geocode=f'4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [3]:
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [4]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,UNEB Bogotá en Puerto Resistencia apoyando a l...,2021-05-27 20:43:50,Bogotá,"Bogotá, D.C., Colombia",1398017124229074945,https://twitter.com/i/web/status/1398017124229...
1,346 Desaparecidos durante Paro Nacional en Co...,2021-05-27 20:42:23,Bogotá,Bogotá,1398016759051997185,https://twitter.com/i/web/status/1398016759051...
2,"#COLOMBIA | Moción de Censura, contra el Minis...",2021-05-27 20:40:45,Bogotá,Pereira-Risaralda-Colombia,1398016344587706372,https://twitter.com/i/web/status/1398016344587...
3,Liked on YouTube: TAMBALEAN PLANES DESESTABILI...,2021-05-27 20:40:06,Bogotá,Bogota,1398016183132172289,https://twitter.com/i/web/status/1398016183132...
4,Alimentos continúan presentando variación de p...,2021-05-27 20:40:00,Bogotá,Eje Cafetero,1398016156594737153,https://twitter.com/i/web/status/1398016156594...


In [5]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [6]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions
0,UNEB Bogotá en Puerto Resistencia apoyando a l...,2021-05-27 20:43:50,Bogotá,"Bogotá, D.C., Colombia",1398017124229074945,https://twitter.com/i/web/status/1398017124229...,UNEB Bogotá en Puerto Resistencia apoyando a l...,[https://t.co/XGDlbDv5cJ],[],[]
1,346 Desaparecidos durante Paro Nacional en Co...,2021-05-27 20:42:23,Bogotá,Bogotá,1398016759051997185,https://twitter.com/i/web/status/1398016759051...,346 Desaparecidos durante Paro Nacional en Co...,[https://t.co/doyCwBjV5q],[],[@dw_espanol]
2,"#COLOMBIA | Moción de Censura, contra el Minis...",2021-05-27 20:40:45,Bogotá,Pereira-Risaralda-Colombia,1398016344587706372,https://twitter.com/i/web/status/1398016344587...,"| Moción de Censura, contra el Ministro de De...",[],"[#COLOMBIA, #Risaralda,]","[@AlejoCorralesCD, @Samy_Merheg,]"
3,Liked on YouTube: TAMBALEAN PLANES DESESTABILI...,2021-05-27 20:40:06,Bogotá,Bogota,1398016183132172289,https://twitter.com/i/web/status/1398016183132...,Liked on YouTube: TAMBALEAN PLANES DESESTABILI...,"[https://t.co/DuppV7r0XV, https://t.co/ygXTA6N...",[],[]
4,Alimentos continúan presentando variación de p...,2021-05-27 20:40:00,Bogotá,Eje Cafetero,1398016156594737153,https://twitter.com/i/web/status/1398016156594...,Alimentos continúan presentando variación de p...,[https://t.co/W97mCB6ZIW],[],[]


In [10]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
pd.set_option('display.max_colwidth', None)

data['sent_vader'] = data['clean'].apply(lambda val: vader.polarity_scores(val)['compound'])
data['sent_textblob'] = data['clean'].apply(lambda val: TextBlob(val).sentiment.polarity)
data[['clean', 'sent_vader', 'sent_textblob']].head()

,clean,sent_vader,sent_textblob
0,"UNEB Bogotá en Puerto Resistencia apoyando a los Jóvenes, entregando una contribución de almuerzo en solidaridad a la resistencia de los Jóvenes en este paro nacional. UNEB Si VALE LA PENA",0.0000,0.0
1,346 Desaparecidos durante Paro Nacional en Colombia: las cifras no cuadran porque el Estado no las busca GRACIAS,-0.5267,0.0
2,"| Moción de Censura, contra el Ministro de Defensa se cayó, porque entre otros, los Senadores de y apoyaron la reprensión y se fueron en contra de la manifestación, dicen jóvenes organizadores del paro Nacional.",0.0000,0.0
3,Liked on YouTube: TAMBALEAN PLANES DESESTABILIZADORES DE PETRO Y DEL COMITÉ NACIONAL DEL PARO,0.4215,0.6
4,Alimentos continúan presentando variación de precios en medio del paro nacional\n\n,0.0000,0.0


###  `[10 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [13]:
data[['clean', 'sent_vader' ]].sort_values('sent_vader').head()

,clean,sent_vader
71,Que viva el paro nacional sin vandalismo y sin que se maten. en Colombia,-0.8020
74,"Atentos,estos son los cómplices de asesinato de jóvenes durante el paro nacional, que le dieron aval al criminal seguir destrozando al país y llenar de terror y miedo a los colombianos,!difundir por favor!!\n \n",-0.7163
88,"No podemos reducir el paro al vandalismo. Debemos rechazar violaciones de DDHH y bloqueos, pero el paro es mucho más Es el estallido social de una juventud frustrada y sin futuro y la oportunidad para dialogar y construir un mejor país. Equivocado titular",-0.7003
69,"Si sachi es buenísimo y si lo traen excelente, pero no por la hinchada, le recuerdo que antes que la hinchada empezará a joder con el no fútbol y si al paro nacional 2 partidos 4 puntos, después el siguiente partido no lo dejaron jugar de local y 4 partidos 1 punto",-0.6808
67,"| Bogotá vivió ayer una de las jornadas con menor participación ciudadana desde que el 28 de abril empezó el paro nacional. \n\nSin embargo, al caer la noche se vivieron horas de disturbios en Kennedy y Usme.",-0.5574


In [14]:
data[['clean', 'sent_vader' ]].sort_values('sent_vader').tail()

,clean,sent_vader
18,Liked on YouTube: TAMBALEAN PLANES DESESTABILIZADORES DE PETRO Y DEL COMITÉ NACIONAL DEL PARO,0.4215
3,Liked on YouTube: TAMBALEAN PLANES DESESTABILIZADORES DE PETRO Y DEL COMITÉ NACIONAL DEL PARO,0.4215
12,"En la coalición de la ""esperanza"" esta cuyos amigotes son los líderes del Comité Nacional del Paro, realmente quieren que haya diálogo y desbloqueos? Nada más peligroso que los políticos que venden odio disfrazado de amor.",0.6124
31,"Bogotá. \nPedir un capuchino y que lo sirvan así, (VPN). VIVA PARO NACIONAL, no tiene precio. 😘😘😘\n\n \n ✨👊🏼💛",0.9136
83,INVITADOS TODOS A EL PRIMER MES DEL PARO NACIONAL❤️💙💛💛,0.9274


In [15]:
data[['clean', 'sent_textblob' ]].sort_values('sent_textblob').head()

,clean,sent_textblob
74,"Atentos,estos son los cómplices de asesinato de jóvenes durante el paro nacional, que le dieron aval al criminal seguir destrozando al país y llenar de terror y miedo a los colombianos,!difundir por favor!!\n \n",-0.625
5,"El pueblo debe comprender que Banda Criminal comité de paro solamente están representando sus sindicatos es decir menos de 0.5% de la población nacional y al sujeto Francisco Maltés y sus compinches poco o nada les importa otros gremios ellos van por lo suyo, así que a trabajar.",-0.400
0,"UNEB Bogotá en Puerto Resistencia apoyando a los Jóvenes, entregando una contribución de almuerzo en solidaridad a la resistencia de los Jóvenes en este paro nacional. UNEB Si VALE LA PENA",0.000
70,No se juega VIVA EL PARO NACIONAL,0.000
69,"Si sachi es buenísimo y si lo traen excelente, pero no por la hinchada, le recuerdo que antes que la hinchada empezará a joder con el no fútbol y si al paro nacional 2 partidos 4 puntos, después el siguiente partido no lo dejaron jugar de local y 4 partidos 1 punto",0.000


In [16]:
data[['clean', 'sent_textblob' ]].sort_values('sent_textblob').tail()

,clean,sent_textblob
88,"No podemos reducir el paro al vandalismo. Debemos rechazar violaciones de DDHH y bloqueos, pero el paro es mucho más Es el estallido social de una juventud frustrada y sin futuro y la oportunidad para dialogar y construir un mejor país. Equivocado titular",0.066667
91,Hoy en dramas de nos quedamos sin bocadillo veleño. \nNi de chiste se les ocurre un titular tipo: la policía asesinó a un civil en Usme durante la jornada de ayer del Paro Nacional.,0.100000
13,"Llevamos varias semanas en las que medios de comunicación, y organizaciones sociales y estatales han denunciado más de 20 casos de violencia sexual en el marco del Paro Nacional 2021. (Abrir hilo para ver todas las imágenes)",0.500000
18,Liked on YouTube: TAMBALEAN PLANES DESESTABILIZADORES DE PETRO Y DEL COMITÉ NACIONAL DEL PARO,0.600000
3,Liked on YouTube: TAMBALEAN PLANES DESESTABILIZADORES DE PETRO Y DEL COMITÉ NACIONAL DEL PARO,0.600000


###  `[10 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [18]:
positivos_vader = data[data['sent_vader'] > 0]
len(positivos_vader)

9

In [19]:
negativos_vader = data[data['sent_vader'] < 0]
len(negativos_vader)

19

In [20]:
neutros_vader = data[data['sent_vader'] == 0]
len(neutros_vader)

72

In [23]:
positivos_textblob = data[data['sent_textblob'] > 0]
len(positivos_textblob)

6

In [25]:
negativos_textblob = data[data['sent_textblob'] < 0]
len(negativos_textblob)

2

In [27]:
neutros_textblob = data[data['sent_textblob'] == 0]
len(neutros_textblob)

92

###  `[5 pts]` : Punto 4: Una visualización sobre sus datos

In [28]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)
data['fecha'] = data.date.dt.floor('T').dt.time

temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por Hora')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [30]:
from collections import Counter
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
3,😘,3
6,💛,3
8,👉,2
10,👇🏽,2
0,👇,1


In [31]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)